# 🔐 URL Security Classifier — Model Training

## Ensemble: XGBoost + DistilBERT (Binary Classification)

**Goal**: Train a high-accuracy model to classify URLs as **Safe** or **Malicious**

**Architecture**:
1. **XGBoost** — 100+ hand-crafted URL features → explainable, fast inference
2. **DistilBERT** — Fine-tuned on raw URL text → learns character-level patterns
3. **Ensemble** — Weighted combination of calibrated probabilities → best of both

**Dataset**: [Kaggle Malicious URLs](https://www.kaggle.com/datasets/sid321axn/malicious-urls-dataset) (~651K URLs → binary)

**Output**: Packaged models ready for server integration

In [ ]:
# ============================================================
# Install Dependencies (run this cell first!)
# ============================================================
!pip install -q kagglehub xgboost optuna scikit-learn transformers accelerate matplotlib seaborn tqdm

import warnings
warnings.filterwarnings('ignore')
print("✅ All packages installed!")

---
## 📊 Section 1: Data Loading & Preprocessing
Download the dataset, explore class distribution, convert to binary (Safe vs Malicious), and split into train/val/test.

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# ── 1.1 Download Dataset ─────────────────────────────────────
import kagglehub

print("📥 Downloading dataset from Kaggle...")
path = kagglehub.dataset_download("sid321axn/malicious-urls-dataset")
print(f"   Downloaded to: {path}")

csv_files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)
csv_file = csv_files[0] if csv_files else os.path.join(path, "malicious_phish.csv")

df = pd.read_csv(csv_file)
print(f"\n✅ Loaded {df.shape[0]:,} URLs  |  Columns: {list(df.columns)}")

# ── 1.2 Explore ──────────────────────────────────────────────
print("\n📊 Original class distribution:")
print(df['type'].value_counts().to_string())

# ── 1.3 Convert to Binary ────────────────────────────────────
df['label'] = (df['type'] != 'benign').astype(int)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
df['type'].value_counts().plot.bar(ax=axes[0], color=['#34C759','#FF3B30','#FF9500','#007AFF'])
axes[0].set_title('Original 4-Class Distribution'); axes[0].set_ylabel('Count')
df['label'].value_counts().rename({0:'Safe', 1:'Malicious'}).plot.bar(ax=axes[1], color=['#34C759','#FF3B30'])
axes[1].set_title('Binary (Safe vs Malicious)'); axes[1].set_ylabel('Count')
plt.tight_layout(); plt.show()

# ── 1.4 Clean ────────────────────────────────────────────────
before = len(df)
df = df.dropna(subset=['url','label']).drop_duplicates(subset=['url'])
print(f"\n🧹 Removed {before - len(df):,} rows → {len(df):,} URLs remaining")

# ── 1.5 Split: 70 / 15 / 15 ─────────────────────────────────
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"\n📊 Split:")
print(f"   Train:      {len(train_df):>8,}")
print(f"   Validation: {len(val_df):>8,}")
print(f"   Test:       {len(test_df):>8,}")

---
## 🌲 Section 2: XGBoost with Feature Engineering

Hand-crafted **100+ features** covering:
- **Length** — URL, domain, path, query, subdomain, TLD lengths
- **Counts** — dots, hyphens, digits, special chars, subdomains, path depth
- **Ratios** — digit/letter/special proportions, domain-to-URL ratio
- **Entropy** — Shannon entropy of domain, path, query, subdomain (randomness detection)
- **Boolean** — IP address, port, HTTPS, hex encoding, punycode, @ symbol
- **TLD** — suspicious/trusted TLD classification
- **Keywords** — phishing, malware, brand impersonation detection
- **Structure** — deep paths, embedded URLs, base64 patterns, shorteners

In [ ]:
import re, math
from urllib.parse import urlparse, parse_qs
from collections import Counter

# ── Keyword / Pattern Dictionaries ────────────────────────────

SUSPICIOUS_TLDS = frozenset({
    "tk","ml","ga","cf","gq","pw","top","xyz","club","work","click","link",
    "surf","buzz","fun","monster","quest","cam","icu","cc","ws","info","biz",
    "su","ru","cn","online","site","website","space","tech","store","stream",
    "download","win","review","racing","cricket","science","party","gdn",
    "loan","men","country","kim","date","faith","accountant","bid","trade","webcam",
})
TRUSTED_TLDS = frozenset({
    "edu","gov","mil","int","ac.uk","gov.uk","edu.au","gov.au",
})
BRAND_KEYWORDS = frozenset({
    "paypal","apple","google","microsoft","amazon","facebook","netflix",
    "instagram","whatsapp","twitter","linkedin","ebay","dropbox","icloud",
    "outlook","office365","yahoo","chase","wellsfargo","bankofamerica",
    "citibank","capitalone","steam","spotify","adobe","coinbase","binance","metamask",
})
PHISHING_KEYWORDS = frozenset({
    "login","signin","sign-in","logon","password","verify","verification",
    "confirm","update","secure","security","account","banking","wallet",
    "suspend","suspended","urgent","expire","unlock","restore","recover",
    "validate","authenticate","webscr","customer","support","helpdesk",
})
MALWARE_KEYWORDS = frozenset({
    "download","free","crack","keygen","patch","serial","warez","torrent",
    "nulled","hack","cheat","generator","install","setup","update","flash",
    "player","codec","driver",
})
URL_SHORTENERS = frozenset({
    "bit.ly","goo.gl","tinyurl.com","ow.ly","t.co","is.gd",
    "buff.ly","adf.ly","j.mp","rb.gy","cutt.ly","tiny.cc",
})
DANGEROUS_EXTS = frozenset({
    ".exe",".dll",".bat",".cmd",".msi",".scr",".pif",".vbs",
    ".js",".jar",".apk",".dmg",".zip",".rar",".7z",".iso",
})

# ── Helper Functions ──────────────────────────────────────────

def calc_entropy(text: str) -> float:
    """Shannon entropy — higher = more random."""
    if not text: return 0.0
    freq = Counter(text.lower())
    n = len(text)
    return -sum((c/n) * math.log2(c/n) for c in freq.values() if c > 0)

def max_run(text: str, cond) -> int:
    """Longest consecutive run of chars matching cond."""
    best = cur = 0
    for ch in text:
        if cond(ch): cur += 1; best = max(best, cur)
        else: cur = 0
    return best

# ── Main Feature Extractor ────────────────────────────────────

def extract_features(url: str) -> dict:
    """Extract 100+ features from a single URL."""
    f = {}
    url = str(url).strip()

    try:
        parsed = urlparse(url if "://" in url else f"http://{url}")
    except Exception:
        return {k: 0 for k in FEATURE_NAMES}

    scheme   = parsed.scheme.lower()
    netloc   = parsed.netloc.lower()
    path     = parsed.path
    query    = parsed.query
    fragment = parsed.fragment

    # Domain cleanup
    netloc_no_port = netloc.split(":")[0] if (":" in netloc and not netloc.startswith("[")) else netloc
    domain = netloc_no_port
    parts  = domain.split(".")
    path_parts = [p for p in path.split("/") if p]
    subdomain  = ".".join(parts[:-2]) if len(parts) > 2 else ""
    tld = parts[-1] if parts else ""
    if len(parts) >= 3 and parts[-2] in ("co","com","org","net","gov","ac","edu"):
        tld = f"{parts[-2]}.{parts[-1]}"

    url_lower  = url.lower()
    path_lower = path.lower()

    # ═══ LENGTH ═══
    f['url_length']         = len(url)
    f['domain_length']      = len(domain)
    f['path_length']        = len(path)
    f['query_length']       = len(query)
    f['fragment_length']    = len(fragment)
    f['subdomain_length']   = len(subdomain)
    f['tld_length']         = len(tld)
    f['longest_domain_part']= max((len(p) for p in parts), default=0)
    f['avg_domain_part_len']= np.mean([len(p) for p in parts]) if parts else 0
    f['longest_path_part']  = max((len(p) for p in path_parts), default=0)
    f['avg_path_part_len']  = np.mean([len(p) for p in path_parts]) if path_parts else 0

    # ═══ COUNTS ═══
    for ch, name in [(".",  "dot"),  ("-", "hyphen"), ("_", "underscore"),
                     ("/",  "slash"),("?", "question"),("=","equals"),
                     ("&",  "amp"),  ("@", "at"),     ("%", "percent"),
                     ("~",  "tilde"),("#", "hash"),   (":", "colon"),
                     (";",  "semicolon")]:
        f[f'{name}_count'] = url.count(ch)

    f['domain_dot_count']    = domain.count(".")
    f['domain_hyphen_count'] = domain.count("-")
    f['domain_digit_count']  = sum(c.isdigit() for c in domain)
    f['subdomain_count']     = max(0, len(parts) - 2)
    f['path_depth']          = len(path_parts)
    f['digit_count']         = sum(c.isdigit() for c in url)
    f['letter_count']        = sum(c.isalpha() for c in url)
    f['uppercase_count']     = sum(c.isupper() for c in url)
    f['special_char_count']  = sum(not c.isalnum() for c in url)

    try:
        qp = parse_qs(query)
        f['query_param_count']     = len(qp)
        f['query_value_total_len'] = sum(len(v) for vals in qp.values() for v in vals)
    except Exception:
        f['query_param_count'] = 0; f['query_value_total_len'] = 0

    # ═══ RATIOS ═══
    ul = max(len(url), 1); dl = max(len(domain), 1)
    f['digit_ratio']         = f['digit_count'] / ul
    f['letter_ratio']        = f['letter_count'] / ul
    f['special_char_ratio']  = f['special_char_count'] / ul
    f['uppercase_ratio']     = f['uppercase_count'] / max(f['letter_count'], 1)
    f['domain_digit_ratio']  = f['domain_digit_count'] / dl
    f['domain_hyphen_ratio'] = f['domain_hyphen_count'] / dl
    f['path_url_ratio']      = f['path_length'] / ul
    f['query_url_ratio']     = f['query_length'] / ul
    f['domain_url_ratio']    = f['domain_length'] / ul

    # ═══ ENTROPY ═══
    f['url_entropy']       = calc_entropy(url)
    f['domain_entropy']    = calc_entropy(domain.replace(".", ""))
    f['path_entropy']      = calc_entropy(path)
    f['query_entropy']     = calc_entropy(query)
    f['subdomain_entropy'] = calc_entropy(subdomain)

    # ═══ BOOLEAN ═══
    f['is_https']                = int(scheme == "https")
    f['is_http']                 = int(scheme == "http")
    f['has_www']                 = int(domain.startswith("www."))
    f['has_port']                = int(":" in netloc and not netloc.startswith("["))
    f['has_at_symbol']           = int("@" in url)
    f['has_double_slash_in_path']= int("//" in path)
    f['has_hex_encoding']        = int(bool(re.search(r"%[0-9a-fA-F]{2}", url)))
    f['has_punycode']            = int("xn--" in domain)
    f['has_ip_address']          = int(bool(re.match(r"^(\d{1,3}\.){3}\d{1,3}$", domain)))
    f['has_hex_ip']              = int(bool(re.match(r"^(0x[0-9a-f]+\.){3}0x[0-9a-f]+$", domain)))
    f['has_ip_like']             = int(domain.replace(".", "").isdigit() and len(domain) > 6)

    # ═══ TLD ═══
    f['is_suspicious_tld'] = int(tld in SUSPICIOUS_TLDS)
    f['is_trusted_tld']    = int(tld in TRUSTED_TLDS)
    f['is_com']            = int(tld == "com")
    f['is_org']            = int(tld == "org")
    f['is_net']            = int(tld == "net")
    f['is_country_tld']    = int(len(tld) == 2 and tld.isalpha())

    # ═══ CHARACTER DISTRIBUTION ═══
    f['max_consec_digits']  = max_run(url, str.isdigit)
    f['max_consec_letters'] = max_run(url, str.isalpha)
    f['max_consec_special'] = max_run(url, lambda c: not c.isalnum())
    vowels = set("aeiou")
    dom_letters = [c for c in domain if c.isalpha()]
    f['domain_vowel_ratio'] = sum(c in vowels for c in dom_letters) / max(len(dom_letters), 1)

    # ═══ KEYWORDS ═══
    f['brand_keyword_count']    = sum(1 for b in BRAND_KEYWORDS if b in url_lower)
    f['has_brand_in_subdomain'] = int(any(b in subdomain.lower() for b in BRAND_KEYWORDS))
    f['phishing_keyword_count'] = sum(1 for k in PHISHING_KEYWORDS if k in url_lower)
    f['malware_keyword_count']  = sum(1 for k in MALWARE_KEYWORDS if k in url_lower)
    f['is_url_shortener']       = int(any(s in netloc for s in URL_SHORTENERS))
    f['has_dangerous_ext']      = int(any(path_lower.endswith(e) for e in DANGEROUS_EXTS))
    f['has_exe']                = int(path_lower.endswith(".exe"))
    f['has_php']                = int(".php" in path_lower)

    # ═══ STRUCTURAL PATTERNS ═══
    f['has_double_letters']  = int(bool(re.search(r"(.)\1", domain)))
    f['has_long_subdomain']  = int(len(subdomain) > 20)
    f['has_deep_path']       = int(len(path_parts) > 5)
    f['has_embedded_url']    = int("http" in path_lower or "www" in path_lower)
    f['has_data_uri']        = int(url_lower.startswith("data:"))
    f['has_javascript']      = int("javascript:" in url_lower)
    f['has_base64']          = int(bool(re.search(r"[A-Za-z0-9+/]{20,}={0,2}", url)))
    f['brand_in_domain']     = int(any(b in domain for b in BRAND_KEYWORDS))
    f['brand_not_registered']= int(
        f['brand_in_domain'] == 1 and not any(
            domain == f"{b}.com" or domain == f"www.{b}.com" for b in BRAND_KEYWORDS
        )
    )

    return f

# Build canonical feature name list
FEATURE_NAMES = list(extract_features("https://www.example.com/path?q=1").keys())
print(f"✅ Feature engineering ready — {len(FEATURE_NAMES)} features per URL")

In [ ]:
from tqdm import tqdm

# ── 2.1 Extract Features from All URLs ───────────────────────
print("⚙️  Extracting features (this takes a few minutes)...\n")

def extract_batch(urls, desc="Extracting"):
    return pd.DataFrame(
        [extract_features(str(u)) for u in tqdm(urls, desc=desc)],
        columns=FEATURE_NAMES
    ).fillna(0).astype(np.float32)

X_train_feat = extract_batch(train_df['url'].tolist(), "  Train")
X_val_feat   = extract_batch(val_df['url'].tolist(),   "  Val")
X_test_feat  = extract_batch(test_df['url'].tolist(),  "  Test")

y_train = train_df['label'].values
y_val   = val_df['label'].values
y_test  = test_df['label'].values

# Class imbalance ratio (used by both XGBoost and DistilBERT later)
scale_pos = float((y_train == 0).sum() / (y_train == 1).sum())

print(f"\n✅ Feature matrices ready:")
print(f"   Train: {X_train_feat.shape}")
print(f"   Val:   {X_val_feat.shape}")
print(f"   Test:  {X_test_feat.shape}")
print(f"   Class ratio (safe/malicious): {scale_pos:.2f}")